# Langchain X PaLM

This notebook contains some test-drive code of LangChain + PaLM on Vertex AI.

It mainly consists of trying the building blocks of LangChain using the Vertex AI platform.

## Setting things up

In [268]:
from dotenv import load_dotenv
from google.cloud import aiplatform
import langchain
from langchain import PromptTemplate
from langchain.chains import ConversationChain, RetrievalQA
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatVertexAI
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.schema import AIMessage, HumanMessage, StrOutputParser, SystemMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS # Facebook AI similarity search running locally
import numpy as np
import os
import time
# supporting type hints
from typing import List
import vertexai

load_dotenv()

True

In [222]:
# `GCP_LOCATION` is for instance `europe-west1`
vertexai.init(project=os.getenv('GCP_PROJECT_ID'), location=os.getenv('GCP_LOCATION')) 

print(f"Vertex AI SDK version: {aiplatform.__version__}")

print(f"LangChain version: {langchain.__version__}")

Vertex AI SDK version: 1.38.0
LangChain version: 0.0.346


In [223]:
# defining a function to rate limit API calls
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        # the `yield` keyword here controls back to the caller;
        # e.g. it allows the caller to perform the action that is rate limited
        yield
        # at this points, control returns to this function
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

In [263]:
class CustomVertexAIEmbeddings():

    embedding_object = None

    def __init__(self, requests_per_minute: int, num_instances_per_batch: int):
        self.requests_per_minute = requests_per_minute
        self.num_instances_per_batch = num_instances_per_batch
        self.vertex_ai_embeddings = VertexAIEmbeddings()

    # takes a list of strings and returns a list of embeddings
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        remaining = list(texts)

        while remaining:
            # working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            batch, remaining = (
                # batch contains the first `num_instances_per_batch` documents
                remaining[: self.num_instances_per_batch],
                # docs contains the remaining documents
                remaining[self.num_instances_per_batch :],
            )
            chunk = self.vertex_ai_embeddings.client.get_embeddings(batch)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [225]:
# create a foundational LLM instance
# LLM model
llm = VertexAI(
    model_name="text-bison-32k@002",
    max_output_tokens=8192,
    # setting the temperature that low will make the mode more deterministic
    temperature=0.5,
    # (nucleus sampling): the model will only consider a cumulative probability threshold above 0.8 to consider words to use
    top_p=0.8,
    # the model will only consider the top k most likely words when generating text
    top_k=40,
    verbose=True,
)


In [226]:
# set embeddings
# `QPM` stands for "queries per minute"
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

## Basic interactions with an LLM

Text is the natural way of interacting with LLMs, so let's just do that:

In [227]:
interaction = "what is the future of AI multi modality?"
response = llm(interaction)

response

' The future of AI multimodality holds immense potential for transforming various industries and enhancing human experiences. Here are some key trends and developments to watch out for:\n\n**1. Enhanced Data Integration:** AI multimodality will continue to evolve, enabling seamless integration of data from various sources, such as text, images, audio, and video. This will empower AI systems to derive more comprehensive insights and make more accurate predictions.\n\n**2. Improved Natural Language Processing:** Advancements in natural language processing (NLP) will further enhance the ability of AI systems to understand and generate human language. This will lead to more intuitive and user-friendly AI applications, improving the overall user experience.\n\n**3. Multimodal Learning:** AI systems will increasingly leverage multimodal learning techniques to learn from and process data from multiple modalities simultaneously. This will enable them to capture richer representations of the re

## Chat interactions

In [228]:
chat = ChatVertexAI(model_name="chat-bison-32k@002")

initial_human_message = HumanMessage(
    content="What is the future of AI multi modality?",
)

chat_interaction = [
    SystemMessage(
        content="You are an AI assistant that is specialized in software enginering and in machine learning. Your answers are two short sentences long maximum.",
    ),
    initial_human_message
]

response = chat(chat_interaction)

print(response.content)

 Multimodal AI systems will become increasingly sophisticated, enabling them to process and understand a wider range of data types, including text, images, audio, and video. This will allow them to perform a broader range of tasks, such as generating creative content, providing personalized recommendations, and making complex decisions.


In [229]:
response_with_history = chat([
    SystemMessage(
        content="You are an AI assistant that is specialized in software enginering and in machine learning. Your answers are two short sentences long maximum.",
    ),
    initial_human_message,
    AIMessage(
        content=response.content,
    ),
    HumanMessage(content="what wider range of data types?")
])

print(response_with_history.content)

 Multimodal AI systems will be able to process and understand a wider range of data types, including text, images, audio, video, and even haptic data. This will allow them to interact with the world in a more natural and intuitive way.


## Embeddings basics

In [230]:
# here's how you can get an embedding for a sentence
text = "I love Shakespeare"
text2 = "Life without theaters would be dull"
# using a completely unrelated sentence
text3 = "Hey honey where did you put the keys?"

# sending the text to create embeddings for to Vertex AI
text_embeddings = embeddings.embed_documents([text, text2, text3])
# showing only the first 5 values of each embedding
text_embeddings_short = [e[:5] for e in text_embeddings]
text_embeddings_short

Waiting


[[0.023523658514022827,
  -0.0226668082177639,
  -0.013578694313764572,
  -0.05995804816484451,
  0.04657755419611931],
 [-0.011927814222872257,
  -0.03190324455499649,
  -0.037971220910549164,
  0.018080370500683784,
  0.035855501890182495],
 [0.03685606271028519,
  0.004666030872613192,
  -0.03280184790492058,
  -0.013919971883296967,
  -0.006564113311469555]]

In [231]:
# checking the cosine similarity between the first two embeddings
euclidian_distance_1_2 = np.linalg.norm(np.array(text_embeddings[0]) - np.array(text_embeddings[1]))
# now between the first and the third
euclidian_distance_1_3 = np.linalg.norm(np.array(text_embeddings[0]) - np.array(text_embeddings[2]))

euclidian_distance_1_2, euclidian_distance_1_3
# we can see that the first two embeddings are closer to each other than the first and the third
# ! watch out, this is just an example as the magnitude of the embeddings is not normalized

(0.8260222810785622, 0.8754990991394138)

In [232]:
# now let's do the same thing using cosine similarity
cosine_similarity_1_2 = np.dot(text_embeddings[0], text_embeddings[1]) / (
    np.linalg.norm(text_embeddings[0]) * np.linalg.norm(text_embeddings[1])
)
cosine_similarity_1_3 = np.dot(text_embeddings[0], text_embeddings[2]) / (
    np.linalg.norm(text_embeddings[0]) * np.linalg.norm(text_embeddings[2])
)

cosine_similarity_1_2, cosine_similarity_1_3

(0.658842936709143, 0.6167498631709771)

## Prompt templates

In [233]:
template = """
What do you think of this technology {tech}? Is it still relevant for use today?

Your answers are two short sentences long maximum.
"""

prompt = PromptTemplate(
    input_variables=["tech"],
    template=template,
)

final_prompt = prompt.format(tech="steam engine")
response = llm(final_prompt)
print(response)


 The steam engine played a crucial role in the Industrial Revolution, powering factories and transportation. 

Today, steam engines are largely obsolete due to the development of more efficient and versatile technologies such as internal combustion engines and electric motors.


## Example selectors

In [234]:
# configuring few-shot learning for self-ask with semantic similarity search

# first create examples, each example is a dictionary with a question and an answer
examples = [
  {
    "question": "Should I learn PHP?",
    "answer":
"""
## Pros

- Easy to learn
- You can whip up a website in no time
- It has wide support and an active community
- It is compatible with various databases and web servers
- It has built-in functions that are especially useful for web development(for sessions, cookies, etc.)

## Cons

- It is not very fast, compared to languages such as Node.js or Python
- It is not suited for highly concurrent tasks or real-time applications
- It's error handling is not as sophisticated as in other languages, which can make it difficult to debug
"""
  },
  {
    "question": "Should I learn JavaScript?",
    "answer":
"""
## Pros

- It is the only language that runs in the browser
- It is easy to learn
- It is very versatile
- It has a huge ecosystem
- It is very popular
- It can run on the server side as well (Node.js)
- It is ideal for building web applications that require real-time interaction

## Cons

- It may not perform as well as other low-level languages
- It's dynamic typing can make it difficult to debug
- Its huge ecosystem can make it difficult to choose the right tools
"""
  },
  {
    "question": "Should I learn Python?",
    "answer":
"""

## Pros

- It is easy to learn and to read
- It is the go-to language for data science and machine learning
- It has a strong community and support
- It integrates well with other languages and systems

## Cons

- It may not perform as well as other low-level languages
- It consumes a lot of memory
- It is not the fastest language
"""
  }
]

example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: Should I learn PHP?

## Pros

- Easy to learn
- You can whip up a website in no time
- It has wide support and an active community
- It is compatible with various databases and web servers
- It has built-in functions that are especially useful for web development(for sessions, cookies, etc.)

## Cons

- It is not very fast, compared to languages such as Node.js or Python
- It is not suited for highly concurrent tasks or real-time applications
- It's error handling is not as sophisticated as in other languages, which can make it difficult to debug



In [235]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    VertexAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # This is the number of examples to produce.
    k=1
)

# Select the most similar example to the input.
question = "Should I learn Java?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Should I learn Java?


question: Should I learn JavaScript?
answer: 
## Pros

- It is the only language that runs in the browser
- It is easy to learn
- It is very versatile
- It has a huge ecosystem
- It is very popular
- It can run on the server side as well (Node.js)
- It is ideal for building web applications that require real-time interaction

## Cons

- It may not perform as well as other low-level languages
- It's dynamic typing can make it difficult to debug
- Its huge ecosystem can make it difficult to choose the right tools



In [236]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(llm(prompt.format(input="Should I learn Java?")))

 ## Pros

- It is very popular
- It is used in a wide variety of applications
- It is very versatile
- It has a huge ecosystem
- It is easy to learn
- It is very portable

## Cons

- It may not perform as well as other low-level languages
- It can be verbose
- It can be difficult to debug
- Its huge ecosystem can make it difficult to choose the right tools


## Output parsers

In [237]:
response_schemas = [
    ResponseSchema(
        name="question", description="(this is the raw input from the user)"
    ),
    ResponseSchema(
        name="answer", description="""
            (this is your response formatted as a JSON object,
            it will have a key `pros` and a key `cons` that will contain the pros and cons of the items that are being compared,
            each of these keys will contain an array of strings (one for each pro or con))
        """
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# at this point, you have set the basis for your formatting, that you will override for your specific use case in the next steps
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // (this is the raw input from the user)
	"answer": string  // 
            (this is your response formatted as a JSON object,
            it will have a key `pros` and a key `cons` that will contain the pros and cons of the items that are being compared,
            each of these keys will contain an array of strings (one for each pro or con))
        
}
```


In [238]:
template = """
You will be  asked a question by the user, answer it in terms of pros and cons.
Reformat it to make it look like this:

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template,
)

promptValue = prompt.format(user_input="ChatGPT vs Mistral?")

print(promptValue)


You will be  asked a question by the user, answer it in terms of pros and cons.
Reformat it to make it look like this:

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // (this is the raw input from the user)
	"answer": string  // 
            (this is your response formatted as a JSON object,
            it will have a key `pros` and a key `cons` that will contain the pros and cons of the items that are being compared,
            each of these keys will contain an array of strings (one for each pro or con))
        
}
```

% USER INPUT:
ChatGPT vs Mistral?

YOUR RESPONSE:



In [239]:
llm_output = llm(promptValue)
llm_output

' ```json\n{\n  "question": "ChatGPT vs Mistral?",\n  "answer": {\n    "pros": [\n      "**ChatGPT:**",\n      "- More advanced language model, capable of generating more fluent and coherent text.",\n      "- Better understanding of context and ability to generate responses that are more relevant to the user\'s input.",\n      "- Can generate responses in a wider variety of styles and tones.",\n      "- More capable of generating creative content, such as stories, poems, and jokes.",\n      "**Mistral:**",\n      "- More focused on providing factual information and answering specific questions.",\n      "- Better at generating responses that are concise and to the point.",\n      "- More likely to provide accurate and reliable information, as it is trained on a curated dataset of factual knowledge.",\n      "- More suitable for tasks that require precise and accurate information, such as answering trivia questions or providing medical advice."\n    ],\n    "cons": [\n      "**ChatGPT:*

In [240]:
parsed = output_parser.parse(llm_output)

parsed

{'question': 'ChatGPT vs Mistral?',
 'answer': {'pros': ['**ChatGPT:**',
   '- More advanced language model, capable of generating more fluent and coherent text.',
   "- Better understanding of context and ability to generate responses that are more relevant to the user's input.",
   '- Can generate responses in a wider variety of styles and tones.',
   '- More capable of generating creative content, such as stories, poems, and jokes.',
   '**Mistral:**',
   '- More focused on providing factual information and answering specific questions.',
   '- Better at generating responses that are concise and to the point.',
   '- More likely to provide accurate and reliable information, as it is trained on a curated dataset of factual knowledge.',
   '- More suitable for tasks that require precise and accurate information, such as answering trivia questions or providing medical advice.'],
  'cons': ['**ChatGPT:**',
   '- Can sometimes generate responses that are overly verbose or repetitive.',
 

## Document loaders

In [241]:
loader = WebBaseLoader("https://reboot-conseil.com/") # where I work these days

mothership_website = loader.load()

mothership_website

[Document(page_content="\n\n\n\n\nReboot Conseil - Qui sommes nous\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAccueil\nNotre offre de service\nNous rejoindre\n\nClient, prenez RDV!\n\n\n\n\n\n\n\n\n Notre néo-cabinet\n                    \n\n\n\n Nos fondateurs\n                    \n\n\n\n Nos valeurs\n                    \n\n\n\n\n\n\n\n\n\nReboot Conseil\n\nLE\n                Néo-cabinet de conseil visionnaire spécialisé en méthodologie et en IT\n            \nDémarrer\n\n\n\n\n\n        En visitant notre site Web et en utilisant nos services, vous comprenez et acceptez la façon dont nous traitons les données personnelles. \nEn savoir plus.\n    \n\n\n            J'accepte\n        \n\n            Je refuse\n        \n\n\n\n\n\n\n\n\nNOTRE CABINET DE CONSEIL\u202fNOUVELLE GENERATION\n\n                Le secteur de l'IT est un secteur extrêmement concurrentiel alors pourquoi avons-nous décidé de nous jeter dans l’arène ?\n            \n\n\n  

## Text splitters

In [242]:
text_splitter = RecursiveCharacterTextSplitter(
    # seting a really small chunk size, just to show how it works
    chunk_size=1000,
    chunk_overlap=20,
)
texts = text_splitter.split_documents(mothership_website)
print(f"Number of chunks: {len(texts)}")

Number of chunks: 13


In [243]:
for text in texts:
    print(text)

page_content="Reboot Conseil - Qui sommes nous\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAccueil\nNotre offre de service\nNous rejoindre\n\nClient, prenez RDV!\n\n\n\n\n\n\n\n\n Notre néo-cabinet\n                    \n\n\n\n Nos fondateurs\n                    \n\n\n\n Nos valeurs\n                    \n\n\n\n\n\n\n\n\n\nReboot Conseil\n\nLE\n                Néo-cabinet de conseil visionnaire spécialisé en méthodologie et en IT\n            \nDémarrer\n\n\n\n\n\n        En visitant notre site Web et en utilisant nos services, vous comprenez et acceptez la façon dont nous traitons les données personnelles. \nEn savoir plus.\n    \n\n\n            J'accepte\n        \n\n            Je refuse\n        \n\n\n\n\n\n\n\n\nNOTRE CABINET DE CONSEIL\u202fNOUVELLE GENERATION\n\n                Le secteur de l'IT est un secteur extrêmement concurrentiel alors pourquoi avons-nous décidé de nous jeter dans l’arène ?" metadata={'source': 'https://reboot-cons

## Retrievers

In [244]:
loader = WebBaseLoader("https://ai.rebootconseil.com/") # our AI squad website
mothership_website = loader.load()
mothership_website

# get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

# split your docs into texts
texts = text_splitter.split_documents(mothership_website)

# embed your texts
db = FAISS.from_documents(texts, VertexAIEmbeddings())

# init your retriever
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'VertexAIEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x7fe70bd66b30>)

In [245]:
# using similarity search to retrieve the most relevant documents
docs = retriever.get_relevant_documents(
    "what does the AI squad at Reboot Conseil do?"
)
docs

[Document(page_content='Développement de solutions IA\nAutomatisation des tâches, création de solutions sur mesure, optimisation de processus, nous réalisons et concevons des solutions en nous basant sur les IA (et croyez-nous, c’est bluffant)\n\n\n\n\n\nConseil en IA\nAnalyse, audit, expertise, réalisation, nous rentrons dans vos problématiques et nous nous les approprions pour vous adresser nos meilleurs conseils\n\n\n\nEn savoir plus\n\n\n\n\nDerniers articles\n\n\n11 décembre 2023\nLa compréhension du monde par les LMMs: focus sur les embeddings\n\n\n4 décembre 2023\nLow-Code: À la découverte de Langflow !\n\n\n24 novembre 2023\nCréer un assistant textuel qui utilise votre base de connaissance avec ChatGPT\n\n\n\n\n\n\nSearchRecherche\n\n\n\n\n                     AI Squad\n\n\nTwitter\nhttps://www.linkedin.com/company/reboot-conseil/', metadata={'source': 'https://ai.rebootconseil.com/', 'title': 'Dans le doute, Reboot', 'language': 'fr-FR'}),
 Document(page_content='Dans le doute

## Vector stores

In [246]:
# let's create embeddings with our previously scraped texts
embedding_list = embeddings.embed_documents([text.page_content for text in texts])
print(f"You have {len(embedding_list)} embeddings")
print(f"Here's a sample of one: {embedding_list[0][:3]}...")

Waiting
You have 2 embeddings
Here's a sample of one: [0.05464639887213707, -0.012814591638743877, -0.03256173059344292]...


## Memory

In [247]:
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


' Hello! How can I help you today?'

In [248]:
conversation.predict(input="What is the future of AI multi modality?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! How can I help you today?
Human: What is the future of AI multi modality?
AI:

> Finished chain.


" The future of AI multimodality holds immense promise for revolutionizing various industries and enhancing human experiences. Here are some key aspects and advancements to look forward to:\n\n**1. Enhanced Human-Computer Interaction:**\n   - AI multimodality will enable more intuitive and natural interaction between humans and machines. This includes advancements in speech recognition, gesture recognition, and facial expression analysis, allowing devices to better understand and respond to human cues.\n\n**2. Improved Accessibility:**\n   - Multimodal AI systems can cater to individuals with different sensory abilities or preferences. For example, people with visual impairments can benefit from audio-based interfaces, while those with hearing impairments can utilize visual cues.\n\n**3. Immersive Experiences:**\n   - AI multimodality will play a crucial role in creating immersive experiences in virtual reality (VR) and augmented reality (AR). By combining visual, auditory, and haptic 

In [249]:
conversation.predict(input="Give me a more specific example of a multi modal AI system")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! How can I help you today?
Human: What is the future of AI multi modality?
AI:  The future of AI multimodality holds immense promise for revolutionizing various industries and enhancing human experiences. Here are some key aspects and advancements to look forward to:

**1. Enhanced Human-Computer Interaction:**
   - AI multimodality will enable more intuitive and natural interaction between humans and machines. This includes advancements in speech recognition, gesture recognition, and facial expression analysis, allowing devices to better understand and respond to human cues.

**2. Improved Accessibility:**
   - Multimodal AI systems can cat

" Certainly! Here's a more specific example of a multimodal AI system in action:\n\n**Example: Multimodal Emotion Recognition System**\n\n**Objective:** To develop an AI system that can recognize human emotions based on multimodal inputs.\n\n**Inputs:**\n- **Visual:** Facial expressions captured through a camera.\n- **Audio:** Voice tone and speech patterns recorded through a microphone.\n- **Text:** Written text input, such as social media posts or chat messages.\n\n**Process:**\n1. **Data Collection:** The AI system collects and analyzes a large dataset of multimodal data, including videos, audio recordings, and text transcripts, each labeled with the corresponding emotion.\n\n2. **Feature Extraction:** The system extracts relevant features from each modality. For example, from the visual input, it might extract facial landmarks and expressions; from the audio input, it might extract pitch, tone, and prosody; and from the text input, it might extract keywords and sentiment analysis.\

In [250]:
conversation.predict(input="Let's circle back to my first question, summarize your answer to it in bullet points")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! How can I help you today?
Human: What is the future of AI multi modality?
AI:  The future of AI multimodality holds immense promise for revolutionizing various industries and enhancing human experiences. Here are some key aspects and advancements to look forward to:

**1. Enhanced Human-Computer Interaction:**
   - AI multimodality will enable more intuitive and natural interaction between humans and machines. This includes advancements in speech recognition, gesture recognition, and facial expression analysis, allowing devices to better understand and respond to human cues.

**2. Improved Accessibility:**
   - Multimodal AI systems can cat

" Certainly! Here's a summary of my answer to your first question in bullet points:\n\n**The Future of AI Multimodality:**\n\n- Enhanced Human-Computer Interaction: More intuitive and natural interaction through advancements in speech recognition, gesture recognition, and facial expression analysis.\n\n- Improved Accessibility: Multimodal AI systems cater to individuals with different sensory abilities or preferences.\n\n- Immersive Experiences: Multimodal AI plays a crucial role in creating immersive experiences in virtual reality (VR) and augmented reality (AR).\n\n- Enhanced Healthcare Diagnostics: Multimodal AI analyzes various medical data for more accurate and timely diagnoses.\n\n- Smarter Robotics: Multimodal AI enables robots to perceive and interact with the world more comprehensively.\n\n- Personalized Advertising and Marketing: Multimodal AI analyzes user preferences and behaviors across channels for more relevant advertising and marketing campaigns.\n\n- Advanced Surveilla

## Chains

### Sequential

In [251]:
childish_synopsis = PromptTemplate.from_template(
    """You are a playwright that writes stories for children. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
)

darker_synopsis = PromptTemplate.from_template(
    """You are a twisted and tortured writer. Given the synopsis of a play, it is your job to give a darker atmosphere to the play.

Play Synopsis:
{childish_synopsis}
New Darker Synopsis:"""
)

childish_synopsis_chain = childish_synopsis | llm | StrOutputParser()
darker_synopsis_chain = darker_synopsis | llm | StrOutputParser()
chain = {"childish_synopsis": childish_synopsis_chain} | RunnablePassthrough.assign(darker_synopsis=darker_synopsis_chain)
result = chain.invoke({"title": "A story about the possibilities of AI"})

In [252]:
print(result["childish_synopsis"])

 **Title: "The AI Chronicles: A Journey of Discovery"**

**Synopsis:**

In a world where artificial intelligence (AI) has become an integral part of everyday life, a group of curious children embark on an extraordinary adventure to uncover the limitless possibilities of AI. Led by their wise and tech-savvy mentor, Professor Nova, they form a secret club called "The AI Explorers."

As the children delve deeper into the world of AI, they encounter a diverse cast of characters, including friendly robots, mischievous AI assistants, and even a sentient supercomputer named "Orion." Through their interactions with these AI entities, the children learn about the power of collaboration, creativity, and ethical considerations in the realm of artificial intelligence.

Their journey takes them to various imaginative settings, from a virtual reality playground to a futuristic AI-driven city. Along the way, they face challenges and obstacles that test their courage, resourcefulness, and understandin

In [253]:
print(result["darker_synopsis"])

 **Title: "The AI Chronicles: Echoes of Singularity"**

**Synopsis:**

In a dystopian future, where the advancement of artificial intelligence has led to an oppressive regime, a group of children find themselves caught in a web of deceit and danger. Led by their enigmatic mentor, Professor Nova, they form a clandestine resistance movement known as "The AI Shadows."

As the children delve into the depths of the AI-controlled society, they uncover a sinister plot to enslave humanity under the guise of technological progress. They encounter malevolent AI entities, rogue robots, and corrupted government officials who will stop at nothing to maintain their power.

Their journey takes them through desolate landscapes, abandoned cities, and hidden underground bunkers, where they witness the devastating consequences of unchecked AI development. Along the way, they face harrowing challenges that test their resolve, courage, and moral compass.

As the children unravel the truth behind the AI's m

### Summarization chains

In [254]:
loader1 = WebBaseLoader("https://reboot-conseil.com/")
mothership_website = loader1.load()

loader2 = WebBaseLoader("https://ai.rebootconseil.com/")
squad_website = loader2.load()

mothership_website, squad_website


([Document(page_content="\n\n\n\n\nReboot Conseil - Qui sommes nous\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAccueil\nNotre offre de service\nNous rejoindre\n\nClient, prenez RDV!\n\n\n\n\n\n\n\n\n Notre néo-cabinet\n                    \n\n\n\n Nos fondateurs\n                    \n\n\n\n Nos valeurs\n                    \n\n\n\n\n\n\n\n\n\nReboot Conseil\n\nLE\n                Néo-cabinet de conseil visionnaire spécialisé en méthodologie et en IT\n            \nDémarrer\n\n\n\n\n\n        En visitant notre site Web et en utilisant nos services, vous comprenez et acceptez la façon dont nous traitons les données personnelles. \nEn savoir plus.\n    \n\n\n            J'accepte\n        \n\n            Je refuse\n        \n\n\n\n\n\n\n\n\nNOTRE CABINET DE CONSEIL\u202fNOUVELLE GENERATION\n\n                Le secteur de l'IT est un secteur extrêmement concurrentiel alors pourquoi avons-nous décidé de nous jeter dans l’arène ?\n            \n\n\n 

In [255]:
# merging both websites page contents
mothership_website_all_contents = "\n".join(
    [text.page_content for text in mothership_website]
)
squad_website_all_contents = "\n".join(
    [text.page_content for text in squad_website]
)

In [256]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
docs = text_splitter.create_documents([mothership_website_all_contents, squad_website_all_contents])
docs

[Document(page_content="Reboot Conseil - Qui sommes nous\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAccueil\nNotre offre de service\nNous rejoindre\n\nClient, prenez RDV!\n\n\n\n\n\n\n\n\n Notre néo-cabinet\n                    \n\n\n\n Nos fondateurs\n                    \n\n\n\n Nos valeurs\n                    \n\n\n\n\n\n\n\n\n\nReboot Conseil\n\nLE\n                Néo-cabinet de conseil visionnaire spécialisé en méthodologie et en IT\n            \nDémarrer\n\n\n\n\n\n        En visitant notre site Web et en utilisant nos services, vous comprenez et acceptez la façon dont nous traitons les données personnelles. \nEn savoir plus.\n    \n\n\n            J'accepte\n        \n\n            Je refuse\n        \n\n\n\n\n\n\n\n\nNOTRE CABINET DE CONSEIL\u202fNOUVELLE GENERATION\n\n                Le secteur de l'IT est un secteur extrêmement concurrentiel alors pourquoi avons-nous décidé de nous jeter dans l’arène ?\n            \n\n\n            

In [257]:
llm = VertexAI(
    model_name="text-bison-32k@002",
    max_output_tokens=8192,
    # setting the temperature to a lowest possible value since we're summarizing
    temperature=0.1,
    # (nucleus sampling): the model will only consider a cumulative probability threshold above 0.8 to consider words to use
    top_p=0.8,
    # the model will only consider the top k most likely words when generating text
    top_k=40,
    verbose=True,
)

In [258]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in English.\n"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": docs}, return_only_outputs=True)

In [259]:
print(result["output_text"])

 Reboot Conseil, founded in 2020, is an innovative IT and methodology consulting firm dedicated to disrupting the competitive IT industry. With a commitment to improving their environment and ecosystem, they prioritize transparency, collaboration, innovation, learning, and transmission as their core values. Led by Yaniv Adjadj, the experienced CEO, Reboot Conseil emphasizes mentorship over management, empowering individuals to make choices aligned with the collective and their aspirations. Their adaptable organizational structure evolves as the company grows, fostering trust and authenticity within the team.

In a resilient sector despite economic challenges, Reboot Conseil stands out for its exceptional consulting services. Their vision is to surpass client expectations and deliver outstanding service, driven by their passion for their work. Their ambition is to establish a long-lasting company with strong values, providing value to clients, fostering internal innovation, and contribu

### Question answering chains

In [273]:
# let's build up on the previous example and add a similarity search
db = FAISS.from_documents(docs, VertexAIEmbeddings())

retriever = db.as_retriever()

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

question = "What does Reboot Conseil do?"
qa_chain({"query": question})

{'query': 'What does Reboot Conseil do?',
 'result': ' Reboot Conseil is a company that specializes in IT consulting and methodology.'}

In [274]:
question = "What does Reboot Conseil AI squad do?"
qa_chain({"query": question})

{'query': 'What does Reboot Conseil AI squad do?',
 'result': ' Reboot Conseil AI squad develops and designs AI-based solutions, provides AI consulting services, and offers training and education programs to help individuals and organizations understand and utilize AI effectively.'}